In [1]:
import pinecone
import os 
from dotenv import load_dotenv


/Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Retrieve Keys

In [2]:
# Env variables
load_dotenv()
# Save it into pinecone
API_KEY = os.environ.get("PINECONE_API_KEY")
ENV = os.environ.get("PINECONE_ENVIRONMENT", "us-west4-gcp-free")

### Init pinecone client

In [3]:
pinecone.init(api_key=API_KEY, environment=ENV)

Create index

In [14]:
#pinecone.create_index("quickstart", dimension=8, metric="euclidean")

In [4]:
pinecone.list_indexes()

[]

Before you can query your index using a client, you must connect to the index.

Use the following commands to connect to your index.

In [30]:
index = pinecone.Index("quickstart")

### Insert the data.

To ingest vectors into your index, use the upsert operation.

The upsert operation inserts a new vector in the index or updates the vector if a vector with the same ID is already present.

The following commands upsert 5 8-dimensional vectors into your index.

In [17]:
# Upsert sample data (5 8-dimensional vectors)
index.upsert([
    ("A", [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
    ("B", [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]),
    ("C", [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]),
    ("D", [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]),
    ("E", [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
])

{'upserted_count': 5}

When upserting larger amounts of data, upsert data in batches of 100 vectors or fewer over multiple upsert requests.

In [31]:
index.describe_index_stats()
# Returns:
# {'dimension': 8, 'index_fullness': 0.0, 'namespaces': {'': {'vector_count': 5}}}

{'dimension': 8,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10005}},
 'total_vector_count': 10005}

Query the index and get similar vectors.

The following example queries the index for the three (3) vectors that are most similar to an example 8-dimensional vector using the Euclidean distance metric specified in step 2 ("Create an index.") above.

In [20]:
index.query(
  vector=[0.3, 0.3, 0.3, 0.3, 0.0, 0.3, 0.3, 0.3],
  top_k=3,
  include_values=True
)
# Returns:
# {'matches': [{'id': 'C',
#               'score': 0.0899999142,
#               'values': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
#              {'id': 'B',
#               'score': 0.109999895,
#               'values': [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]},
#              {'id': 'D',
#               'score': 0.230000019,
#               'values': [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]}],
#  'namespace': ''}

{'matches': [{'id': 'C',
              'score': 0.0899999142,
              'values': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
             {'id': 'B',
              'score': 0.109999895,
              'values': [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]},
             {'id': 'D',
              'score': 0.230000019,
              'values': [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]}],
 'namespace': ''}

## Batching upsertr

In [22]:
import random
import itertools

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

vector_dim = 8
vector_count = 10000

# Example generator that generates many (id, vector) pairs
example_data_generator = map(lambda i: (f'id-{i}', [random.random() for _ in range(vector_dim)]), range(vector_count))

# Upsert data with 100 vectors per upsert request
for ids_vectors_chunk in chunks(example_data_generator, batch_size=100):
    index.upsert(vectors=ids_vectors_chunk)  # Assuming `index` defined elsewhere

## Sending upserts in parallel

By default, all vector operations block until the response has been received. But using our client they can be made asynchronous. For the Batching Upserts example this can be done as follows:

In [24]:
# Upsert data with 100 vectors per upsert request asynchronously
# - Create pinecone.Index with pool_threads=30 (limits to 30 simultaneous requests)
# - Pass async_req=True to index.upsert()
with pinecone.Index('example-index', pool_threads=30) as index:
    # Send requests in parallel
    async_results = [
        index.upsert(vectors=ids_vectors_chunk, async_req=True)
        for ids_vectors_chunk in chunks(example_data_generator, batch_size=100)
    ]
    # Wait for and retrieve responses (this raises in case of error)
    [async_result.get() for async_result in async_results]

## Inserting vectors with metadata

You can insert vectors that contain metadata as key-value pairs.

You can then use the metadata to filter for those criteria when sending the query. Pinecone will search for similar vector embeddings only among those items that match the filter. For more information, see: Metadata Filtering.

In [32]:
index.upsert([
    ("A", [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], {"genre": "comedy", "year": 2020}),
    ("B", [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2], {"genre": "documentary", "year": 2019}),
    ("C", [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3], {"genre": "comedy", "year": 2019}),
    ("D", [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4], {"genre": "drama"}),
    ("E", [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], {"genre": "drama"})
])

{'upserted_count': 5}

## Upserting vectors with sparse values

Sparse vector values can be upserted alongside dense vector values.

## Delete index

In [36]:
pinecone.delete_index("quickstart")